In [48]:
import xmltodict 
import json
import os

pokedex_path = '../raw/XmlDump/DexEntries.json'
pokedex_data_dir = '../Version20/Pokedex/'

In [11]:
book_raw = json.loads(open(pokedex_path).read())
len(book_raw)
len(os.listdir(pokedex_data_dir))
cnt = 0
for raw in book_raw:
    flag = False
    for fname in os.listdir(pokedex_data_dir):
        json_data = json.loads(open(pokedex_data_dir+'/'+fname).read())
        n = json_data['Name']
        if n == raw['Name']:
            cnt+=1
            flag = True
            break
    if not flag: print(raw['Name'])
print(cnt)

Necrozma Dawn Wings
1019


In [52]:
book_raw = json.loads(open(pokedex_path).read())
book_keyed = {}
for raw in book_raw:
    book_keyed[raw['Name']] = raw


for fname in os.listdir(pokedex_data_dir):
    # fname = 'Charizard.json'
    if '.json' not in fname: continue
    path = pokedex_data_dir+fname
    data = json.loads(open(path).read())
    new = book_keyed[data['Name']]
    if new['Name'] == 'Egg': continue
    try:
        data["DexCategory"] = new.get('Category',"")
        data['Height'] = {}
        data['Height']['Deimeters'] = int(new['AverageHeight']['@Value'])
        data['Height']['Meters'] = round(int(new['AverageHeight']['@Value'])*0.1, 2)
        data['Height']['Feet'] = round(int(new['AverageHeight']['@Value'])*0.3280839895, 1)
        data['Weight'] = {}
        data['Weight']['Hectograms'] = int(new['AverageWeight']['@Value'])
        data['Weight']['Kilograms'] = round(int(new['AverageWeight']['@Value'])*0.1, 2)
        data['Weight']['Pounds'] = round(int(new['AverageWeight']['@Value'])*0.2204622622, 1)
        data["DexDescription"] = new.get('DexDescription',"")
        data['Baby'] = True if new.get('IsBaby') == "true" else False
        data['PrimaryEggGroup'] = new['PrimaryEggGroup']
        data['SecondaryEggGroup'] = new['SecondaryEggGroup'] if type(new['SecondaryEggGroup']) == str else ""
        
        data['BookImageName'] = new['PrimaryImage']['@DisplayName']
        if new.get('ShinyImage'): data['BookShinyImageName'] = new['ShinyImage']['@DisplayName']
        
        if new['MegaEvolutions']:
            data['MegaEvolutions'] = []
            extract = new['MegaEvolutions']['MegaEvolutionEntry']
            if type(extract) == dict: extract = [extract]
            for evo in extract:
                data['MegaEvolutions'].append(
                    {"Name":evo['TargetEvolution']['@DisplayName'], "Item":evo['Item']['@DisplayName']})
    except:
        print("ERROR: "+ new['Name'])
        print(new)
        raise
        # raise err
    
    # print(json.dumps(data))
    # break
    open(path,'w').write(json.dumps(data, indent=4))